In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

import pandas as pd
import re
import time
import numpy as np

In [13]:
def scrape_articles(journal, start, end, times):
    
    email = 'xianlin.ding@sciencespo.fr'
    password = '099115Keep12?'
    
    # go into the Eruopresse website
    url = "https://acces-distant.sciencespo.fr/fork?https://nouveau.europresse.com/access/ip/default.aspx?un=politique2T_1"
    driver = webdriver.Chrome()
    driver.get(url)
    driver.find_element(By.ID, 'username').send_keys(email)
    driver.find_element(By.ID, 'password').send_keys(password)
    time.sleep(3)
    driver.find_element(By.CLASS_NAME, 'btn-submit').click()
    
    # choose the criteria of searching
    driver.find_element(By.CLASS_NAME, 'lnk-text').click()
    time.sleep(3)
    driver.find_element(By.ID, 'specific-sources-rd').click()
    time.sleep(10)
    
    # choose the journal
    driver.find_element(By.ID, journal).click()
    
    # key words
    driver.find_element(By.XPATH, '//textarea').send_keys(keywords) 
    
    # choose the searching period
    Select(driver.find_element(By.ID, 'DateFilter_DateRange')).select_by_value('10')

    Select(driver.find_elements(By.CLASS_NAME, 'year')[1]).select_by_value(start[0])
    Select(driver.find_elements(By.CLASS_NAME, 'year')[2]).select_by_value(end[0])

    Select(driver.find_elements(By.CLASS_NAME, 'month')[1]).select_by_value(start[1])
    Select(driver.find_elements(By.CLASS_NAME, 'month')[2]).select_by_value(end[1])

    Select(driver.find_elements(By.CLASS_NAME, 'day')[1]).select_by_value(start[2])
    Select(driver.find_elements(By.CLASS_NAME, 'day')[2]).select_by_value(end[2])
    
    driver.find_element(By.ID, 'btnSearch').click()
    
    # scroll page
    for i in range(times):
        temp_height=0
        while True:
            driver.execute_script("window.scrollBy(0,1000)")
            check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
            if check_height==temp_height:
                break
            temp_height=check_height
        
        time.sleep(5)
        print('scrolling ', i+1, ' times' )
        
    # get web contents
    web_content = driver.page_source
    obj = BeautifulSoup(web_content,'lxml').body
    obj = obj.aside.ul
    obj = obj.find_all(name='div', attrs={"class":"docListItem msDocItem"})
    
    # start to build data
    info = pd.DataFrame(columns = ['date', 'publication', 
                                   'words', 'author',
                                   'title', 'intro'])
    
    i=0
    for obj_i in obj:
        
        obj_i = obj_i.div.next_sibling.next_sibling
        
        # publication
        obj_i_1 = obj_i.div
        obj_i_1 = obj_i_1.find_all(name='span', attrs={"class":"source-name"})[0]
        obj_i_1 = str(obj_i_1)
        publication_i = re.findall(r"<span class=\"source-name\">(.+?)</span>", obj_i_1)[0]
        
        # title
        obj_i_2 = obj_i.div.next_sibling.next_sibling
        title_i = obj_i_2.div.a
        title_i = str(title_i)
        title_i = re.findall(r">(.+?)</a>", title_i)[0]
        
        # date and words
        obj_i_3 = obj_i_2.div.next_sibling.next_sibling 
        
        date_words_i = obj_i_3.div
        date_words_i = str(date_words_i)
        date_i = re.findall(r"<span class=\"details\">(.+?)<span class=", date_words_i)[0]
        date_i = date_i.replace(' ', '')
        
        words_i = re.findall(r"</span>(.+?)</span>", date_words_i)[0]
        words_i = words_i.replace(' ', '')
        
        # authors and introduction
        obj_i_4 = obj_i_3.div.next_sibling.next_sibling
        obj_i_4 = str(obj_i_4)
        
        try:
            obj_i_4 = obj_i_4.split('<span class="doclist-author">')[1]
            author_i = obj_i_4.split('</span>')[0]
            intro_i = obj_i_4.split('</span>')[1]
        
        except:
            author_i = None
            intro_i = obj_i_4.split('</span> </span>')[1]
    
        intro_i = intro_i.split('</div>')[0]
        intro_i = re.sub(r"<([a-z]+)>",'',intro_i)
        intro_i = re.sub(r"</([a-z]+)>",'',intro_i)
        intro_i = re.sub('\u200a','',intro_i)
        intro_i = re.sub('-','',intro_i)
        
        info.loc[i] = [date_i, publication_i, 
                       words_i, author_i,
                       title_i, intro_i]
        i = i+1
    
    return info

In [14]:
## data 1 : keywords - climat
journal = 'sf_247'  # le monde
keywords = 'climat'  # keywords
start = ['2022','10','1'] # searching period
end = ['2022','12','31']
times = 50 # scrolling times

In [15]:
data = scrape_articles(journal, start, end, times)

scrolling  1  times
scrolling  2  times
scrolling  3  times
scrolling  4  times
scrolling  5  times
scrolling  6  times
scrolling  7  times
scrolling  8  times
scrolling  9  times
scrolling  10  times
scrolling  11  times
scrolling  12  times
scrolling  13  times
scrolling  14  times
scrolling  15  times


In [16]:
data

,date,publication,words,author,title,intro
0,2022-11-05,Le Monde,3064mots,Audrey Garric,"Antonio Guterres, voix ardente pour le climat",Les formules chocs sur le climat sont dev...
1,2022-12-10,Le Monde,1159mots,Rémi Barroux,Climat : les promesses non tenues du budget 2023,"... en font le constat. L’une, rédigée p..."
2,2022-10-28,Le Monde,1288mots,Audrey Garric,"Pour le climat, « une année gâchée »",... un nouvel avertissement pour les gouv...
3,2022-11-05,Le Monde,1280mots,Mathilde Gérard,L’agriculture et l’alimentation oubliées,Achaque conférence mondiale sur le climat...
4,2022-10-22,Le Monde,1262mots,Propos recueillis par Audrey Garric (à Paris) ...,"« Face au réchauffement, chaque Etat doit sort...",... qui rend plus difficile la lutte cont...
...,...,...,...,...,...,...
395,2022-12-20,Le Monde,812mots,Aline Leclerc,"Dans l’Hexagone, les fiches de paie courent ap...",... salaires. Qui vont se retrouver payés...
396,2022-11-05,Le Monde,1157mots,Rémi Barroux,Un tiers des glaciers du Patrimoine mondial co...,"... ’adaptation au changement climatique, ..."
397,2022-11-12,Le Monde,580mots,None,Faire face à la difficulté de la transition éc...,... se trouvent les adeptes d’une déc...
398,2022-11-16,Le Monde,1011mots,Richard Schittly,"Près de Lyon, l’« immense souffrance » d’un ce...",... L’agence évoque « des relations conf...
